In [31]:
import os

images=[]
lable=[]

lables=['Bishop','King','Knight','Pawn','Queen','Rook']
a=0
for i in os.listdir(r"C:\Users\ASUS\OneDrive\Desktop\code\datasets\Chess"):
    c=0
    path=os.path.join(r"C:\Users\ASUS\OneDrive\Desktop\code\datasets\Chess",i)
    for j in os.listdir(path):
        if c>=76:
            break
        images.append(os.path.join(path,j))
        lable.append(lables[a])
        c+=1
    a+=1

In [2]:
len(images)

456

In [27]:
len(lable)

456

In [28]:
for i in lables:
    print(lable.count(i))

76
76
76
76
76
76


In [3]:
import pandas as pd
from PIL import Image
import numpy as np

In [4]:
imagearray=[]
for i in images:
    acimg=Image.open(i)
    acimg=acimg.resize((224,224))
    imgarray=np.array(acimg)
    imgarray=imgarray/255.0
    imagearray.append(imgarray)


In [12]:
imagearray[0]

array([[[0.48627451, 0.35686275, 0.22745098],
        [0.48627451, 0.35686275, 0.22745098],
        [0.48235294, 0.35294118, 0.22352941],
        ...,
        [0.49019608, 0.36862745, 0.29019608],
        [0.49411765, 0.37254902, 0.29411765],
        [0.49411765, 0.37254902, 0.29411765]],

       [[0.48235294, 0.35294118, 0.23137255],
        [0.48235294, 0.35294118, 0.22745098],
        [0.47843137, 0.34901961, 0.21960784],
        ...,
        [0.4627451 , 0.34117647, 0.25882353],
        [0.46666667, 0.34509804, 0.2627451 ],
        [0.46666667, 0.34509804, 0.2627451 ]],

       [[0.47058824, 0.34901961, 0.22745098],
        [0.47058824, 0.34901961, 0.22745098],
        [0.4745098 , 0.34509804, 0.22352941],
        ...,
        [0.43529412, 0.31372549, 0.23137255],
        [0.43921569, 0.31764706, 0.23529412],
        [0.43921569, 0.31764706, 0.23529412]],

       ...,

       [[0.94901961, 0.83137255, 0.54901961],
        [0.95294118, 0.83137255, 0.55294118],
        [0.96470588, 0

In [32]:
dtf={
    # "X":imagearray,
    "X":images,
    "Y":lable
}
df=pd.DataFrame(dtf)
df=df.sample(frac=1)
dt=pd.get_dummies(df["Y"],dtype=float,)
df= pd.concat([df,dt], axis=1)
df = df.drop('Y', axis=1)

In [33]:
df.head()

,X,Bishop,King,Knight,Pawn,Queen,Rook
333,C:\Users\ASUS\OneDrive\Desktop\code\datasets\C...,0.0,0.0,0.0,0.0,1.0,0.0
322,C:\Users\ASUS\OneDrive\Desktop\code\datasets\C...,0.0,0.0,0.0,0.0,1.0,0.0
228,C:\Users\ASUS\OneDrive\Desktop\code\datasets\C...,0.0,0.0,0.0,1.0,0.0,0.0
440,C:\Users\ASUS\OneDrive\Desktop\code\datasets\C...,0.0,0.0,0.0,0.0,0.0,1.0
87,C:\Users\ASUS\OneDrive\Desktop\code\datasets\C...,0.0,1.0,0.0,0.0,0.0,0.0


In [34]:
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest=train_test_split(df['X'],df[lables],random_state=69,test_size=0.3)
xtest,xval,ytest,yval=train_test_split(xtest,ytest,random_state=69,test_size=0.5)

In [21]:
import torch
from torch.utils.data import DataLoader,Dataset
from torchvision import transforms

In [36]:
ransform=transforms.Compose([
    transforms.Resize((128,128)),
    transforms.ToTensor(),
    transforms.ConvertImageDtype(torch.float)
])

In [43]:
df.iloc[0,0]

'C:\\Users\\ASUS\\OneDrive\\Desktop\\code\\datasets\\Chess\\Queen\\00000044.jpg'

In [ ]:
class customdataset(Dataset):
    def __init__(self,dataset,transform=None):
        self.df=dataset
        self.transform=transform
    
    def __len__(self):
        return self.df.shape[0]
    
    def __getitem__(self, index):
        imgpath=self.df.iloc[index,0]
        
      

In [ ]:
print()

i
i
      
